# **_Sentiment Analysis_**

### Here is everything you are going to need to run this notebook

* nltk.download('all')
* pip install pyenchant 
---

There some files that have the results of the cleaning ready.If you want to use them leave the below variable as is.

In [ ]:
use_files=1


The first thing that we are going to do is load all the files that we need.

The twitters datasets first.

In [ ]:
import pandas as pd

dataset_gold = pd.read_csv("../twitter_data/SemEval2017_task4_subtaskA_test_english_gold.txt", encoding='utf8',delimiter='\t',header=None);
dataset_train = pd.read_csv("../twitter_data/train2017.tsv", encoding='utf8',delimiter='\t',header=None);
dataset_test = pd.read_csv("../twitter_data/test2017.tsv", encoding='utf8',delimiter='\t',header=None);


And now the lexicons

In [ ]:
from read_files import read_lexica
all_lex=[] #list of dictionaries
all_lex.append(read_lexica("../lexica/affin/affin.txt"))
all_lex.append(read_lexica("../lexica/emotweet/valence_tweet.txt"))
all_lex.append(read_lexica("../lexica/generic/generic.txt"))
all_lex.append(read_lexica("../lexica/nrc/val.txt"))
all_lex.append(read_lexica("../lexica/nrctag/val.txt"))
all_lex.append(read_lexica("../lexica/sentric/senticnet5.txt"))


Some libraries that we are going to need below are here

In [ ]:
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))  

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

from read_files import read_dict
from fix_tweets import clean_tw

import enchant
d = enchant.Dict("en_US")

import pickle

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score

import numpy as np
import matplotlib.pyplot as plt
import random

from PIL import Image
from os import path ,getcwd
from wordcloud import WordCloud
from colour_wc import random_color_func


This is were the cleaning is.If you want to skip this part and use the files set the use_flies variable to 1.

In [ ]:
y2 = dataset_train.iloc[:, 2]
y3 = dataset_train.iloc[:, 3]
train_data=list(map(list, zip(y2,y3))) #list of tweets and values

fixed_tweets,hash_tags,name_tags=clean_tw(train_data)

words=read_dict()
import csv        
if use_files==0:
    for tweet in fixed_tweets:
        tw=tweet[0]
        tw=tknzr.tokenize(tw)
        tw1=[]
        for word in tw:
            word=word.lower()
            tw1.append(stemmer.stem(word))

        tweet[0] = " ".join(tw1)
    
    with open('resources/twitter_fixed.tsv', 'w', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        for tweet in fixed_tweets:
            tsv_output.writerow([tweet[0]])


### Bag of words


In [ ]:
dataset_fix_train = pd.read_csv('resources/twitter_fixed.tsv', encoding='utf8',delimiter='\t',header=None);

corpus=[]
for i in range(dataset_fix_train.shape[0]):
    tweet = dataset_fix_train.iloc[i, 0]
    corpus.append(str(tweet))

test_tweets=[]   
for i in range(dataset_test.shape[0]):
    tweet = dataset_test.iloc[i, 3]
    test_tweets.append(tweet) 

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words=nltk_stopwords ,max_features=1000)


X = vectorizer.fit_transform(corpus).toarray()
y = dataset_train.iloc[:, 2]

classifier = GaussianNB()
classifier.fit(X, y)

with open('bof_uni.pkl', 'wb') as handle:
    pickle.dump(classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('bof_uni.pkl', 'rb') as handle:
    classifier = pickle.load(handle)
    
X_test= vectorizer.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)
y_test=dataset_gold.iloc[:, 1]



This are the results of bag of words

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

For the bag of words above we used unigrams to get those results.In sentiment analysis this is not the best approach.We will now use bigrams and see how our results change.

In [ ]:
vectorizer_bigram = CountVectorizer(stop_words=nltk_stopwords ,max_features=1000,ngram_range=(2, 2))
X = vectorizer_bigram.fit_transform(corpus).toarray()
classifier = GaussianNB()
classifier.fit(X, y)

with open('bof_bi.pkl', 'wb') as handle:
    pickle.dump(classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('bof_bi.pkl', 'rb') as handle:
    classifier = pickle.load(handle)

X_test= vectorizer_bigram.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)

In [ ]:
The results this time

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print("F1_score with average [None,macro,micro,weighted]")
print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

This time the results seem to be worse.This is not what we expected.This might be because we didnt have enought tweets to train under sets of words.

We will try one last time with trigrams so we can catch cases like "not very good"

In [ ]:
vectorizer_trigram = CountVectorizer(stop_words=nltk_stopwords ,max_features=1000,ngram_range=(3, 3))
X = vectorizer_trigram.fit_transform(corpus).toarray()
classifier = GaussianNB()
classifier.fit(X, y)

with open('bof_tri.pkl', 'wb') as handle:
    pickle.dump(classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('bof_tri.pkl', 'rb') as handle:
    classifier = pickle.load(handle)

X_test= vectorizer_trigram.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)

The results for trigrams

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print("F1_score with average [None,macro,micro,weighted]")
print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

This is better than bigram but still worse than unigram

### Tf-Idf 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=nltk_stopwords, strip_accents="ascii", lowercase=True,max_features=1000)

X = vectorizer.fit_transform(corpus).toarray()
y = dataset_train.iloc[:, 2]

classifier = GaussianNB()
classifier.fit(X, y)

X_test= vectorizer.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)
y_test=dataset_gold.iloc[:, 1]




Results of Tf-idf

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

Again we will try to usr bigrams to see the difference

In [ ]:
vectorizer = TfidfVectorizer(stop_words=nltk_stopwords, strip_accents="ascii", lowercase=True,max_features=1000,ngram_range=(2, 2))

X = vectorizer.fit_transform(corpus).toarray()
y = dataset_train.iloc[:, 2]

classifier = GaussianNB()
classifier.fit(X, y)

X_test= vectorizer.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)
y_test=dataset_gold.iloc[:, 1]

Results

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

Trigrams

In [ ]:
vectorizer = TfidfVectorizer(stop_words=nltk_stopwords, strip_accents="ascii", lowercase=True,max_features=1000,ngram_range=(3, 3))

X = vectorizer.fit_transform(corpus).toarray()
y = dataset_train.iloc[:, 2]

classifier = GaussianNB()
classifier.fit(X, y)

X_test= vectorizer.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)
y_test=dataset_gold.iloc[:, 1]

In [ ]:
Results

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

### Word Embeddings

In [ ]:
from Word2Vec import WordEmbeddingsVectorize

WordEmbeddingsVectorize(fixed_tweets,size=300,pkl_filename"resources/word_embed_pickle.pkl")

### Data Analysis

First we need to load the test file to a list 

In [ ]:
y2=dataset_gold.iloc[:, 1]
y3 = dataset_test.iloc[:, 3]
test_data=list(map(list, zip(y2,y3))) 
  
fixed_tweets,hash_tags,name_tags=clean_tw(train_data)
fixed_tweets1,hash_tags1,name_tags1=clean_tw(test_data)

fixed_tweets+=fixed_tweets1
hash_tags+=hash_tags1
name_tags+=name_tags1
all_data=train_data+test_data


Most used words in tweets

In [ ]:
wd_text=""

for tweet_info in fixed_tweets:
    tweet_info[0]=tknzr.tokenize(tweet_info[0].lower())
    wd_text+=" ".join(tweet_info[0])

d = path.dirname(__file__) if "__file__" in locals() else getcwd()
mask = np.array(Image.open(path.join(d, "twitter_mask.png")))

nltk_WC = WordCloud(background_color="white",max_words=500, mask=mask, stopwords=nltk_stopwords, margin=10,
               random_state=1).generate(wd_text)

plt.figure(figsize=[7,7])
plt.imshow(nltk_WC.recolor(color_func=random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis("off")
plt.show()
nltk_WC.to_file("twitter_general_WC.png")

Most used words in positive tweets

In [ ]:
wd_text=""

for i in range(len(fixed_tweets)):
    if all_data[i][0]=="positive":       
        wd_text+=" ".join(fixed_tweets[i][0])

mask = np.array(Image.open(path.join(d, "twitter_mask.png")))

nltk_WC = WordCloud(background_color="white",max_words=500, mask=mask, stopwords=nltk_stopwords, margin=10,
               random_state=1).generate(wd_text)

plt.figure(figsize=[7,7])
plt.imshow(nltk_WC.recolor(color_func=random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis("off")
plt.show()
nltk_WC.to_file("twitter_positive_WC.png")

Most used words in negative tweets

In [ ]:
wd_text=""

for i in range(len(fixed_tweets)):
    if all_data[i][0]=="negative":       
        wd_text+=" ".join(fixed_tweets[i][0])

mask = np.array(Image.open(path.join(d, "twitter_mask.png")))

nltk_WC = WordCloud(background_color="white",max_words=500, mask=mask, stopwords=nltk_stopwords, margin=10,
               random_state=1).generate(wd_text)

plt.figure(figsize=[7,7])
plt.imshow(nltk_WC.recolor(color_func=random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis("off")
plt.show()
nltk_WC.to_file("twitter_negative_WC.png")

Most used words in neutral tweets

In [ ]:
wd_text=""

for i in range(len(fixed_tweets)):
    if all_data[i][0]=="neutral":       
        wd_text+=" ".join(fixed_tweets[i][0])

mask = np.array(Image.open(path.join(d, "twitter_mask.png")))

nltk_WC = WordCloud(background_color="white",max_words=500, mask=mask, stopwords=nltk_stopwords, margin=10,
               random_state=1).generate(wd_text)

plt.figure(figsize=[7,7])
plt.imshow(nltk_WC.recolor(color_func=random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis("off")
plt.show()
nltk_WC.to_file("twitter_neutral_WC.png")

Most used tags

In [ ]:

wd_text=""
wd_text=" ".join(name_tags)

mask = np.array(Image.open(path.join(d, "twitter_mask.png")))

nltk_WC = WordCloud(background_color="white",max_words=500, mask=mask, stopwords=nltk_stopwords, margin=10,
               random_state=1).generate(wd_text)

plt.figure(figsize=[7,7])
plt.imshow(nltk_WC.recolor(color_func=random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis("off")
plt.show()
nltk_WC.to_file("twitter_tags_WC.png")

Most used hash tags

In [ ]:
wd_text=""
wd_text=" ".join(hash_tags)
mask = np.array(Image.open(path.join(d, "twitter_mask.png")))

nltk_WC = WordCloud(background_color="white",max_words=500, mask=mask, stopwords=nltk_stopwords, margin=10,
               random_state=1).generate(wd_text)

plt.figure(figsize=[7,7])
plt.imshow(nltk_WC.recolor(color_func=random_color_func, random_state=3),
           interpolation="bilinear")
plt.axis("off")
plt.show()
nltk_WC.to_file("twitter_hash_WC.png")

#### General info

In [ ]:
all_tweets_num=len(fixed_tweets)
all_tweets_name_tagg=0
all_tweets_hash_tagg=0
all_tweets_link=0

for tweet in fixed_tweets:
    if len(tweet[1])!=0:
        all_tweets_hash_tagg+=1
    if len(tweet[2])!=0:
        all_tweets_name_tagg+=1   
    if len(tweet[3])!=0:
        all_tweets_link+=1   

        
print("Out of the " +str(all_tweets_num)+" tweets "+str(all_tweets_hash_tagg)
      +" used hash tags "+str(all_tweets_name_tagg)+
      " tagged someone and "+str(all_tweets_name_tagg)+" used links");

Below we try to find mistakes in tweets.This is not very accurate around (60% accuracy)

In [2]:
mistakes=0
for tweet in fixed_tweets:
    tw=tknzr.tokenize(tweet[0])
    for word in tw:
        wd=word.lower()
        if not d.check(wd):
            if wd not in words:
                mistakes+=1
                break
print("Out of the " +str(all_tweets_num)+" tweets "+str(mistakes)+ " had mistakes")


NameError: name 'fixed_tweets' is not defined